# Trade Visualizer

In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [124]:
st = pd.read_csv('../data/generated-statements.csv')

st = st.drop(st.index[26:32]) # remove sus activity
st = st.iloc[::-1].reset_index(drop=True) # chronological order

In [125]:
# INPUT Definitions:
# price = price of usdt for 1 crypto at date
# executed = amount in crypto bought/sold
# amount = amount in usdt bought/sold

# OUTUT Definitions:
# starting_amt = the amount of capital at the moment of the first transaction of each trade
# gain_loss = the gain/loss the trade was responsible for (note: starting_amt+gain_loss does NOT 
# necessarily equal the amount of capital at the end of the trade, because trades in other assets could
# have taken place.)
# buy_date = time of the first long/short transaction
# sell_date = time of the last liquidate transaction

st

,Date(UTC),Pair,Side,Price,Executed,Amount,Fee
0,2021-07-17 21:18:52,CELRUSDT,BUY,0.02361,710.6000000000CELR,16.77726600USDT,0.7106000000CELR
1,2021-07-17 21:18:52,FETUSDT,SELL,0.21647,77.4000000000FET,16.75477800USDT,0.0167547800USDT
2,2021-07-17 21:20:40,FETUSDT,BUY,0.21652,77.5000000000FET,16.78030000USDT,0.0775000000FET
3,2021-07-17 21:20:40,CELRUSDT,SELL,0.02357,709.8000000000CELR,16.72998600USDT,0.0167299900USDT
4,2021-07-17 21:40:35,CELRUSDT,BUY,0.02362,708.3000000000CELR,16.73004600USDT,0.7083000000CELR
5,2021-07-17 21:40:35,FETUSDT,SELL,0.21566,77.5000000000FET,16.71365000USDT,0.0167136500USDT
6,2021-07-18 20:03:22,CELRUSDT,SELL,0.02586,709.3000000000CELR,18.34249800USDT,0.0183425000USDT
7,2021-07-18 20:03:22,FETUSDT,BUY,0.23781,77.4000000000FET,18.40649400USDT,0.0774000000FET
8,2021-07-18 20:05:21,FETUSDT,BUY,0.23684,70.6000000000FET,16.72090400USDT,0.0706000000FET
9,2021-07-18 20:05:21,CELRUSDT,SELL,0.02585,646.2000000000CELR,16.70427000USDT,0.0167042700USDT


In [126]:
# find pairs & trade type
# (since we are not given which two cryptos are pairs, and there can be multiple pairs)

# ⚠️⚠️ assumes the statement df has no unliquidated trades at the start,
# therefore the first two unique cryptos are a pair,
# and the first set of trades for each pair is a long-short, not a liquidation ⚠️⚠️

# if the above assumptions are not true, need to 1) provide pairs_dict (e.g. {'FETUSDT': 'CELRUSDT', ...})
# and 2) provide type_dict, aka the type of trade of the *FIRST* trade for each crypto
# (e.g. {'FETUSDT': 'LONGSHORT', 'CELRUSDT': 'LONGSHORT', 'BTCUSDT': 'LIQUIDATE', ...})

def find_trades(statement, pairs_dict={}, type_dict={}):
    out = statement.copy()
    out['Group_ID'] = np.nan
    out['Type'] = np.nan
    
    #finding pairs_dict
    if not pairs_dict:
        i = 0
        while i < out.shape[0]:
            
            c1 = out['Pair'][i]
            c2 = c1
            j = 1
            while c2 == c1 and i+j < out.shape[0]:
                c2 = out['Pair'][i+j]
                j += 1
                
            if c1 in pairs_dict.keys() or c1 in pairs_dict.values():
                i += 1
                continue
                
            # note if c1==c2 at end of list, should have already been added to pairs_dict
            if c1 == c2: raise ValueError('Alone pair at end of list')

            pairs_dict[c1] = c2
            
            #not j+1 because j already +=1 at end of while loop
            i += j
    
    # filling the other pair in the dictionary so there is both A:B and B:A
    # pairs_dict.update(dict((v,k) for k,v in pairs_dict.items()))
    
    # after found pairs, label transactions with their "group id" corresponding to one trade action ie, liquidate or long-short
    # kinda inefficient
    curr_group_id = 0
    for p1, p2 in pairs_dict.items():
        
        pair_isolated = out[out['Pair'].isin([p1, p2])]
        curr_type = type_dict[p1] if type_dict else 'LONGSHORT'
        curr_side = {}
        
        if (pair_isolated['Pair'][0] == p1 and pair_isolated['Side'][0] == 'BUY') or \
           (pair_isolated['Pair'][0] == p2 and pair_isolated['Side'][0] == 'SELL'):
            curr_side = {p1: 'BUY', p2: 'SELL'}
        else:
            curr_side = {p1: 'SELL', p2: 'BUY'}
        
        for index, row in pair_isolated.iterrows():
#             print(f'curr_side: {curr_side}')
#             print(f'current c: {row.Pair}, current side: {curr_side[row.Pair]}')
#             print(f'from data: {row.Side}')
#             print()
            
            if row['Side'] != curr_side[row['Pair']]:
                # change current side (ex. from buy to sell)
                l_curr_side = [(k,v) for k,v in curr_side.items()]
                new_curr_side = {l_curr_side[0][0]: l_curr_side[1][1],
                                 l_curr_side[1][0]: l_curr_side[0][1]}
                curr_side = new_curr_side.copy()
                curr_type = 'LONGSHORT' if curr_type == 'LIQUIDATE' else 'LIQUIDATE'
                curr_group_id += 1
                
            out.loc[index, 'Type'] = curr_type
            out.loc[index, 'Group_ID'] = curr_group_id
            
    return out, pairs_dict

In [143]:
def remove_ticker(s):
    return float(''.join(i for i in s if i.isdigit() or i in '.'))

def get_fee_usdt(row):
    fee = row['Fee']
    if fee.endswith("USDT"):
        return remove_ticker(fee)
    return remove_ticker(fee) * row['Price']

def generate_performance(statement, pairs_dict, init_amt):
    trades = pd.DataFrame(columns = ['Asset_A', 'Asset_B', 'Starting_Amt', 'Gain_Loss', 'Buy_Date', 'Sell_Date'])
    
    # removing rows of pairs that start with liquidate or end with longshort
    trimmed_st = statement.copy()
    for p1, p2 in pairs_dict.items():
        pair_isolated = trimmed_st[trimmed_st['Pair'].isin([p1, p2])]
        
        for index, row in pair_isolated.iterrows():
            if row['Type'] == 'LONGSHORT': break
            trimmed_st = trimmed_st.drop(i)
        for index, row in pair_isolated.iloc[::-1].iterrows():
            if row['Type'] == 'LIQUIDATE': break
            trimmed_st = trimmed_st.drop(i)
            
    trimmed_st = trimmed_st.reset_index(drop=True)
        
    # getting trade ids
    trimmed_st['Trade_ID'] = np.nan
    trimmed_st['Trade_Order'] = np.nan
    
    curr_trade_id = -1
    curr_trade_order = 0
    curr_state = 'NONE' #'HOLD' = awaiting liquidate, 'NONE' = awaiting new longshort
    
    for p1, p2 in pairs_dict.items():
        pair_isolated = trimmed_st[trimmed_st['Pair'].isin([p1, p2])]
        
        for index, row in pair_isolated.iterrows():
            if curr_state == 'NONE' and row['Type'] == 'LONGSHORT':
                curr_state = 'HOLD'
                curr_trade_id += 1
                curr_trade_order = 0
            elif curr_state == 'HOLD' and row['Type'] == 'LIQUIDATE':
                curr_state = 'NONE'

            trimmed_st.loc[index, 'Trade_ID'] = curr_trade_id
            trimmed_st.loc[index, 'Trade_Order'] = curr_trade_order
            curr_trade_order += 1
    
    # iteratively getting trades
    curr_amt = init_amt
    
    for index, row in trimmed_st.iterrows():
        
        curr_id = row['Trade_ID']
        curr_gain_loss = 0
        
        if row['Side'] == 'BUY':
            curr_gain_loss = -remove_ticker(row['Amount']) - (get_fee_usdt(row))
        else:
            curr_gain_loss = remove_ticker(row['Amount']) - (get_fee_usdt(row))
            
        if curr_id in trades.index:
            cumulative_gain_loss = trades['Gain_Loss'][curr_id]
            cumulative_gain_loss += curr_gain_loss
            
            trades.loc[curr_id, 'Gain_Loss'] = cumulative_gain_loss
            trades.loc[curr_id, 'Sell_Date'] = row['Date(UTC)']
            
        else:
            asset_a = ''
            asset_b = ''
            if row['Pair'] in pairs_dict.keys():
                asset_a = row['Pair']
                asset_b = pairs_dict[asset_a]
            elif row['Pair'] in pairs_dict.values():
                asset_a = list(pairs_dict.keys())[list(pairs_dict.values()).index(row['Pair'])]
                asset_b = row['Pair']
            
            new_row = pd.Series({'Asset_A':asset_a, 
                                 'Asset_B':asset_b, 
                                 'Starting_Amt': curr_amt,
                                 'Gain_Loss': curr_gain_loss,
                                 'Buy_Date': row['Date(UTC)'],
                                 'Sell_Date': row['Date(UTC)']
                                }, name = curr_id)
            
            trades = trades.append(new_row)
            
        curr_amt += curr_gain_loss
    return trades, trimmed_st

In [147]:
st2, pairs = find_trades(st)
trades, trimmed_st = generate_performance(st2, pairs, 100)

In [148]:
trimmed_st

,Date(UTC),Pair,Side,Price,Executed,Amount,Fee,Group_ID,Type,Trade_ID,Trade_Order
0,2021-07-17 21:18:52,CELRUSDT,BUY,0.02361,710.6000000000CELR,16.77726600USDT,0.7106000000CELR,0.0,LONGSHORT,0.0,0.0
1,2021-07-17 21:18:52,FETUSDT,SELL,0.21647,77.4000000000FET,16.75477800USDT,0.0167547800USDT,0.0,LONGSHORT,0.0,1.0
2,2021-07-17 21:20:40,FETUSDT,BUY,0.21652,77.5000000000FET,16.78030000USDT,0.0775000000FET,1.0,LIQUIDATE,0.0,2.0
3,2021-07-17 21:20:40,CELRUSDT,SELL,0.02357,709.8000000000CELR,16.72998600USDT,0.0167299900USDT,1.0,LIQUIDATE,0.0,3.0
4,2021-07-17 21:40:35,CELRUSDT,BUY,0.02362,708.3000000000CELR,16.73004600USDT,0.7083000000CELR,2.0,LONGSHORT,1.0,0.0
5,2021-07-17 21:40:35,FETUSDT,SELL,0.21566,77.5000000000FET,16.71365000USDT,0.0167136500USDT,2.0,LONGSHORT,1.0,1.0
6,2021-07-18 20:03:22,CELRUSDT,SELL,0.02586,709.3000000000CELR,18.34249800USDT,0.0183425000USDT,3.0,LIQUIDATE,1.0,2.0
7,2021-07-18 20:03:22,FETUSDT,BUY,0.23781,77.4000000000FET,18.40649400USDT,0.0774000000FET,3.0,LIQUIDATE,1.0,3.0
8,2021-07-18 20:05:21,FETUSDT,BUY,0.23684,70.6000000000FET,16.72090400USDT,0.0706000000FET,3.0,LIQUIDATE,1.0,4.0
9,2021-07-18 20:05:21,CELRUSDT,SELL,0.02585,646.2000000000CELR,16.70427000USDT,0.0167042700USDT,3.0,LIQUIDATE,1.0,5.0


In [145]:
trades

,Asset_A,Asset_B,Starting_Amt,Gain_Loss,Buy_Date,Sell_Date
0.0,CELRUSDT,FETUSDT,100,-0.139844,2021-07-17 21:18:52,2021-07-17 21:20:40
1.0,CELRUSDT,FETUSDT,99.860156,-0.200644,2021-07-17 21:40:35,2021-07-18 20:05:21
2.0,CELRUSDT,FETUSDT,99.659512,-1.324573,2021-07-19 14:12:58,2021-07-22 20:06:00
3.0,CELRUSDT,FETUSDT,98.334939,1.981686,2021-07-24 10:52:59,2021-07-24 14:14:03
